In [126]:
import numpy as np
import logging

In [316]:
class Markowitz():
    
    "Will remain a static class and take updates"
    
    def __init__(self,r, r_cov):
        """
        Purpose of thie class is to generate an optimal portfolio weighting based on a given mean
        and covariance matrix
        """
        self.r = r
        self.r_cov = r_cov
        
    def normal_update(self,desired_r,r,r_cov):
        """
        Update function will be called when a covariance and return matrix are given to be processed.
        
        Returns:
            (strat_alloc (np.array (n,),strat_var (float))
        """
        self.r = r
        self.r_cov = r_cov
        
        strat_alloc = self.design_portfolio_for_return(desired_r)
        strat_var = self.variance_of_strategy(strat_alloc)
        
        return (strat_alloc ,strat_var)
    
    def multi_update(self,desired_rs,r,r_cov):
        """
        Update function will be called when a covariance and return matrix are given to be processed.
        
        Returns:
            (strat_alloc (np.array (n,n) ,strat_var (np.array (n,))
        """
        self.r = r
        self.r_cov = r_cov
        
        strat_allocs = self.design_portfolios_for_return(desired_rs)
        strat_vars = np.array([self.variance_of_strategy(strat_alloc) for strat_alloc in strat_allocs])
        
        return (strat_allocs ,strat_vars)
        
    def design_portfolio_for_return(self,r,reinvert=True):
        """
        Given a specific return, we design a portfolio allocation in order to achieve that mean. We additionally return
        the variance associated with that return. The notation for this derivation is consistent with notes that will be
        included in this file.
        
        """
        logging.info("Calculating Optimal Portfolio")
        e = np.ones(self.r.shape)
        
        #Will automatically handle errors
        if reinvert:
            self.inv_cov = self.invert_covariance_matrix()
        
        #Calculating entries in C
        c_r=np.dot(self.inv_cov,self.r)
        c_e=np.dot(self.inv_cov,e)
        
        c_rr= np.dot(self.r,c_r)
        c_re= np.dot(self.r,c_e)
        c_er= np.dot(e,c_r)
        c_ee= np.dot(e,c_e)
        
        det_C = c_ee*c_rr-c_re*c_er
        
        C_inv = np.array([[c_ee,-c_re],[-c_er,c_rr]])/det_C
        
        #Deriving the Lagrangian Multipliers
        lamb, mu = np.dot(C_inv,np.array([r,1]))
        
        #Optimal Asset Allocation
        omega = lamb*c_r+mu*c_e
        
        return omega
    
    def design_portfolios_for_return(self,rs,reinvert=True):
        """
        Uses the two fund theorem and vectorized operations in order to quickly create a 
        range of different portfolios to run.
        """
        logging.info("Calculating Optimal Portfolios")
        e = np.ones(self.r.shape)
        
        #Will automatically handle errors
        if reinvert:
            self.inv_cov = self.invert_covariance_matrix()
        
        #Calculating the two asset allocations for optimal
        v_1=np.dot(self.inv_cov,e) #min variance
        v_2=np.dot(self.inv_cov,self.r) #max return
        
        #Calculating entries in C
        c_r=v_2
        c_e=v_1
        
        c_rr= np.dot(self.r,c_r)
        c_re= np.dot(self.r,c_e)
        c_er= np.dot(e,c_r)
        c_ee= np.dot(e,c_e)
        
        det_C = c_ee*c_rr-c_re*c_er
        alphas = [c_ee*(c_rr-c_er*r)/det_C for r in rs]
        
        #Two identical funds
        w_1=v_1/c_ee
        w_2=v_2/c_er
        
        omegas = np.array([alpha*w_1+(1-alpha)*w_2 for alpha in alphas])
        
        return omegas
    
    def invert_covariance_matrix(self):
        """
        Will invert covariance matrices 
        """
        logging.info("Attempting to Invert Matrix... ")
        try:
            return np.linalg.inv(self.r_cov)
        except:
            return self.handle_covariance_singularity()
        
    def variance_of_strategy(self,omega, reinvert=False):
        """
        Will always return the variance of the most current strategy. Will allow us to quantify risk.
        """
        logging.info("Calculating Variance of Strategy")
        if reinvert:
            self.inv_cov = self.invert_covariance_matrix()
        
        return np.dot(omega,np.dot(self.r_cov,omega))
            
    def handle_covariance_singularity(self,epsilon=1e-8):
        logging.error("Singular Matrix, perturbing it...")
        self.r_cov+=epsilon*np.eye(self.r_cov.shape[0])
        return self.invert_covariance_matrix()
    

In [317]:
mark = Markowitz(np.array([0.1,0.05,0.06,0.03,0.02,0.03,0.06,0.03,0.02,0.03]),np.random.randn(10,10))

In [329]:
mark = Markowitz(np.array([0.1,0.05,0.06,0.03,0.02,0.03,0.06,0.03,0.02,0.03]),np.zeros((10,10)))

In [330]:
v1 = mark.design_portfolio_for_return(0.2)

ERROR:root:Singular Matrix, perturbing it...


In [331]:
np.dot(mark.r,v1)

0.2000000000000001

In [332]:
mark.variance_of_strategy(v1)

4.493761140819965e-08

In [303]:
%timeit v1 = mark.design_portfolio_for_return(0.2)

33.8 µs ± 437 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Attempt to use the two fund design portfolio

In [304]:
v2 = mark.design_portfolios_for_return([0.2,0.3])

In [305]:
np.dot(v2,mark.r)

array([0.2, 0.3])

In [306]:
%timeit v2 = mark.design_portfolios_for_return([0.2,0.3])

37.9 µs ± 1.51 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Testing Update Scheme

In [307]:
mark.normal_update(0.3,np.array([0.1,0.05,0.06,0.03,0.02,0.03,0.06,0.03,0.02,0.03]),np.random.randn(10,10))

(array([ -6.32349994,  -1.88736701,  13.21044225, -11.63985919,
          3.28141062, -16.60331397,  13.27771958,   3.90112051,
          1.14384129,   2.63950586]), -106.78808038253595)

In [308]:
mark.multi_update([0.1,0.11,0.12],np.array([0.1,0.05,0.06,0.03,0.02,0.03,0.06,0.03,0.02,0.03]),np.random.randn(10,10))

(array([[ -0.4665493 ,   0.51134135,   0.22699229,   6.22151297,
          -1.31321181,  -1.39585644,  -0.04105544,   1.02128907,
          -7.37214001,   3.60767731],
        [ -0.60426879,   0.58965009,   0.22117727,   7.50423167,
          -1.57487721,  -1.74129274,  -0.03553866,   1.19028409,
          -8.91878832,   4.3694226 ],
        [ -0.74198829,   0.66795884,   0.21536225,   8.78695037,
          -1.83654261,  -2.08672904,  -0.03002188,   1.3592791 ,
         -10.46543662,   5.13116789]]),
 array([-2.31224475, -2.58436593, -2.80769303]))

In [310]:
%timeit mark.normal_update(0.3,np.array([0.1,0.05,0.06,0.03,0.02,0.03,0.06,0.03,0.02,0.03]),np.random.randn(10,10))

44.5 µs ± 975 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [309]:
%timeit mark.multi_update([0.1,0.11,0.12],np.array([0.1,0.05,0.06,0.03,0.02,0.03,0.06,0.03,0.02,0.03]),np.random.randn(10,10))

59.2 µs ± 1.9 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Questions:

- What are limitations on the cost of executing/making trades
- How fast does this need to run
- How much are we allowed to leverage/short?
- Is there going to be a risk free asset?